### Задача 1: Написание простых SQL-запросов (3б всего, по 0.3б за задание)


В задаче 1 предлагается рассмотреть известную базу данных HR Oracle (ссылки на этой строке для общего развития, не обязательны): https://docs.oracle.com/en/database/oracle/oracle-database/12.2/comsc/HR-sample-schema-table-descriptions.html#GUID-506C25CE-FA5D-472A-9C4C-F9EF200823EE, https://github.com/oracle/db-sample-schemas/tree/main/human_resources

- Скрипты для создания рассматриваемых таблиц и заполнения их данными для разных СУБД: https://www.sqltutorial.org/sql-sample-database/

- Необходимо написать 10 SQL-запросов (каждый по 0.3б), каждый из которых **независимо от предыдущих** будет решать описанную задачу.

- В качестве решения необходимо написать SQL-запрос после соответствующего условия.

- Укажите в конце этой строки СУБД, которую вы используете: (например, MySQL)
- 
  sqlite

1.1. Таблица Employees. Получить список всех сотрудников, у которых зарплата находится в промежутке от 8000 до 9000 (включительно) и/или кратна 1000

In [ ]:
SELECT *
FROM Employees
WHERE (salary BETWEEN 8000 AND 9000) OR (salary % 1000 = 0);

1.2. Таблица Employees. Получить первое трёхзначное число телефонного номера сотрудника, если его номер в формате ХХХ.ХХХ.ХХХХ

In [ ]:
SELECT 
    SUBSTR(phone_number, 1, 3) AS first_three_digits
FROM Employees
LIMIT 1;

1.3. Таблица Departments. Получить первое слово из имени департамента для тех, у кого в названии больше одного слова

In [ ]:
SELECT SUBSTR(department_name, 1, INSTR(department_name, ' ') - 1) AS first_word
FROM Departments
WHERE INSTR(department_name, ' ') > 0;

1.4. Таблица Employees. Получить список всех сотрудников, которые пришли на работу в первый день месяца (любого)

In [ ]:
SELECT *
FROM Employees
WHERE STRFTIME('%d', hire_date) = '01';

1.5. Таблица Employees. Получить уровень зарплаты каждого сотрудника: Меньше 5000 считается Low level, Больше или равно 5000 и меньше 10000 считается Normal level, Больше или равно 10000 считается High level

In [ ]:
SELECT employee_id, first_name, last_name, salary,
       CASE
           WHEN salary < 5000 THEN 'Low level'
           WHEN salary >= 5000 AND salary < 10000 THEN 'Normal level'
           ELSE 'High level'
       END AS salary_level
FROM Employees;

1.6. Таблица Employees. Получить список department_id и округленную среднюю зарплату работников в каждом департаменте.

In [ ]:
SELECT department_id, ROUND(AVG(salary)) AS avg_salary
FROM Employees
GROUP BY department_id;

1.7. Таблицы Employees, Departments, Locations, Countries, Regions. Получить список регионов и количество сотрудников в каждом регионе

In [ ]:
SELECT r.region_name, COUNT(e.employee_id) AS employee_count
FROM Employees e
JOIN Departments d ON e.department_id = d.department_id
JOIN Locations l ON d.location_id = l.location_id
JOIN Countries c ON l.country_id = c.country_id
JOIN Regions r ON c.region_id = r.region_id
GROUP BY r.region_name;

1.8. Таблица Employees. Показать всех менеджеров, которые имеют в подчинении больше шести сотрудников

In [ ]:
SELECT manager_id, COUNT(employee_id) AS subordinates_count
FROM Employees
GROUP BY manager_id
HAVING COUNT(employee_id) > 6;

1.9. Таблицы Employees, Departments. Показать все департаменты, в которых работают больше пяти сотрудников

In [ ]:
SELECT d.department_name, COUNT(e.employee_id) AS employee_count
FROM Employees e
JOIN Departments d ON e.department_id = d.department_id
GROUP BY d.department_name
HAVING COUNT(e.employee_id) > 5;

1.10. Таблица Employees. Получить список сотрудников с зарплатой большей средней зарплаты всех сотрудников.

In [ ]:
SELECT *
FROM Employees
WHERE salary > (SELECT AVG(salary) FROM Employees);

### Задача 2: Асинхронная работа с SQLite (2 б)


Результатом должен быть работающий код на Python, выполняющий все шаги из задания.
1. Создайте базу данных `library.db` с таблицей `books`:
   - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
   - `title` (TEXT),
   - `author` (TEXT),
   - `year` (INTEGER).
2. Асинхронно добавьте три книги:
   - `"Book A"`, автор: `Author 1`, год: 2001;
   - `"Book B"`, автор: `Author 2`, год: 1999;
   - `"Book C"`, автор: `Author 3`, год: 2015.
3. Асинхронно получите список всех книг, опубликованных после 2000 года.

In [1]:
pip install aiosqlite

Note: you may need to restart the kernel to use updated packages.


In [1]:
import nest_asyncio
nest_asyncio.apply()

In [3]:
import asyncio
import aiosqlite

async def create_table():
    async with aiosqlite.connect('library.db') as db:
        await db.execute('''CREATE TABLE IF NOT EXISTS books (
                id INTEGER PRIMARY KEY AUTOINCREMENT,
                title TEXT,
                author TEXT,
                year INTEGER
            )''')
        await db.commit()

async def insert_books():
    async with aiosqlite.connect('library.db') as db:
        await db.execute("INSERT INTO books (title, author, year) VALUES (?, ?, ?)", ("Book A", "Author 1", 2001))
        await db.execute("INSERT INTO books (title, author, year) VALUES (?, ?, ?)", ("Book B", "Author 2", 1999))
        await db.execute("INSERT INTO books (title, author, year) VALUES (?, ?, ?)", ("Book C", "Author 3", 2015))
        await db.commit()

async def get_books_after_2000():
    async with aiosqlite.connect('library.db') as db:
        async with db.execute("SELECT * FROM books WHERE year > 2000") as cursor:
            return await cursor.fetchall()

async def main():
    await create_table()
    await insert_books()
    books = await get_books_after_2000()
    for book in books:
        print(book)

asyncio.run(main())

(1, 'Book A', 'Author 1', 2001)
(3, 'Book C', 'Author 3', 2015)


### Задача 3: Использование оконных функций (2б)

Результатом должен быть работающий код на Python, выполняющий все шаги из задания.
1. Создайте базу данных `sales.db` с таблицей `sales`:
   - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
   - `region` (TEXT),
   - `employee` (TEXT),
   - `amount` (INTEGER).
2. Вставьте данные:
   - `North`, `Alice`, 500;
   - `North`, `Bob`, 300;
   - `South`, `Charlie`, 700;
   - `South`, `David`, 400;
   - `North`, `Eve`, 200.
3. Напишите SQL-запросы с оконными функциями для:
   - Вычисления общего объёма продаж (`amount`) по каждому региону.
   - Вычисления ранга (`rank`) сотрудника в своём регионе на основе суммы продаж.

**Ожидаемый результат:**
```
('North', 'Alice', 500, 1000, 1)
('North', 'Bob', 300, 1000, 2)
('North', 'Eve', 200, 1000, 3)
('South', 'Charlie', 700, 1100, 1)
('South', 'David', 400, 1100, 2)
```

In [7]:
import sqlite3

conn = sqlite3.connect('sales.db')
cursor = conn.cursor()
cursor.execute('''CREATE TABLE IF NOT EXISTS sales (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        region TEXT,
        employee TEXT,
        amount INTEGER)''')
conn.commit()
sales_data = [('North', 'Alice', 500), ('North', 'Bob', 300), ('South', 'Charlie', 700), ('South', 'David', 400), ('North', 'Eve', 200)]
cursor.executemany('INSERT INTO sales (region, employee, amount) VALUES (?, ?, ?)', sales_data)
conn.commit()
cursor.execute('''SELECT region, employee, amount,
           SUM(amount) OVER (PARTITION BY region) AS total_amount,
           RANK() OVER (PARTITION BY region ORDER BY amount DESC) AS rank
    FROM sales''')
for i in cursor.fetchall():
    print(i)
conn.close()

('North', 'Alice', 500, 1000, 1)
('North', 'Bob', 300, 1000, 2)
('North', 'Eve', 200, 1000, 3)
('South', 'Charlie', 700, 1100, 1)
('South', 'David', 400, 1100, 2)


### Задача 4: Работа с ORM: связь многие-ко-многим (3б)

В этом задании всё должно быть сделано с помощью ORM SQLAlchemy или аналогов в Python. Использование чего-либо, где вы просто пишете обычный SQL-запрос, оценивается в 0 баллов за задание.
1. Создайте базу данных `school.db` с таблицами:
   - `students`:
     - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
     - `name` (TEXT, NOT NULL).
   - `courses`:
     - `id` (INTEGER, PRIMARY KEY, AUTOINCREMENT),
     - `title` (TEXT, UNIQUE, NOT NULL).
   - Связующая таблица `student_courses`:
     - `student_id` (INTEGER, ForeignKey(`students.id`)),
     - `course_id` (INTEGER, ForeignKey(`courses.id`)).
2. Добавьте студентов и курсы:
   - Студенты: `Alice`, `Bob`, `Charlie`.
   - Курсы: `Math`, `Physics`, `Chemistry`.
3. Запишите данные о том, кто посещает какие курсы:
   - `Alice` посещает `Math` и `Physics`.
   - `Bob` посещает `Physics` и `Chemistry`.
   - `Charlie` посещает все три курса.
4. Напишите запросы:
   - Вывести всех студентов с их курсами.
   - Найти всех студентов, которые посещают `Physics`.

**Ожидаемый вывод**
```
Alice: Physics, Math
Charlie: Physics, Chemistry, Math
Bob: Physics, Chemistry
Студенты, посещающие Physics:
Alice
Bob
Charlie
```

In [11]:
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table
from sqlalchemy.orm import declarative_base, relationship, sessionmaker

Base = declarative_base()

student_course = Table(
    'student_course', Base.metadata,
    Column('student_id', Integer, ForeignKey('students.id')),
    Column('course_id', Integer, ForeignKey('courses.id'))
)

class Student(Base):
    __tablename__ = 'students'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False)
    courses = relationship('Course', secondary=student_course, back_populates='students')

class Course(Base):
    __tablename__ = 'courses'
    id = Column(Integer, primary_key=True)
    title = Column(String, unique=True, nullable=False)
    students = relationship('Student', secondary=student_course, back_populates='courses')

engine = create_engine('sqlite:///school.db')
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)
session = Session()

alice = Student(name='Alice')
bob = Student(name='Bob')
charlie = Student(name='Charlie')

math = Course(title='Math')
physics = Course(title='Physics')
chemistry = Course(title='Chemistry')

alice.courses.extend([math, physics])
bob.courses.extend([physics, chemistry])
charlie.courses.extend([math, physics, chemistry])
session.add_all([alice, bob, charlie, math, physics, chemistry])
session.commit()
students = session.query(Student).all()
for student in students:
    print(f"{student.name}: {', '.join([course.title for course in student.courses])}")
physics_students = session.query(Student).join(student_course).join(Course).filter(Course.title == 'Physics').all()
print("Студенты, посещающие Physics:")
for student in physics_students:
    print(student.name)
session.close()

Alice: Physics, Math
Charlie: Chemistry, Physics, Math
Bob: Chemistry, Physics
Студенты, посещающие Physics:
Alice
Bob
Charlie
